In [ ]:
# 1. batch_size :  The number of samples to train use in one iteration
# 2. epochs : The number of times the network goes through the enitre training dataset.
# 3. Sample_size : [speculation] The number of images[random noise] used in training for generator.
# 4. nz :[speculation] input feature size for each sample[image] in above sample_size.
# 5. k : hyper-parameter
# 6. device : utilizing GPU if available 

## Dependencies

1. Tqdm :  provides progress bar.
2. Transform : provides common image transform algorithms.
3. make_grid : Make a grid of images.
4. DataLoader: helps in importing training and test sets 

In [ ]:
import torch 
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0)


Tensor:

    1. Tensors are n-dimnesional matrix or vertices.
    2. Pytorch use tensor operations.

Batches:

    1. Batches are the amount of training samples the model performs operation during a single execution.
    2. Batch values are usually chosen to be multiples of 2 (32,64,128,256,512).


# Dataset

    1. The Dataset is downloaded from the MNIST website . Furtherthe data is divided
       into batches , converted from PIL to tensors using DataLoaders.
    2. Dataloader encloses a iterable wrapper over the tensor object using which we
       can access each batch.
    3. Structure : 469 batches -> each containg 128 images -> each image  [1,28,28]

In [ ]:
loss_func = nn.BCEWithLogitsLoss()
epochs = 200
batch_size = 128
lr = 0.0001
z_dim = 64
device = 'cuda'

In [ ]:
dataloader = DataLoader( MNIST("/work/basicGAN/GAN_cadmus/input",download = True,transform = transforms.ToTensor() )
                ,batch_size = batch_size , shuffle = True)

In [ ]:
def show_tensor_images(image_tensor,num_images=25,size = (1,28,28)):

    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [ ]:
def create_gan(input_dim,output_dim):
    
    return nn.Sequential(

        nn.Linear(input_dim,output_dim)
        nn.BatchNorm1d(output_dim)
        nn.LeakyReLU(inplace = False)
    )

SyntaxError: invalid syntax (<ipython-input-44-fa0097b9607f>, line 6)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a5eebfae-950c-452c-bcd1-02b41e6a5850' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>